In [1]:
library(parallel)
library(stringr)

In [2]:
vcf_dir  = '/nfs/lab/projects/pbmc_snATAC/pipeline/snp_array/final_vcfs/'
homedir  = '/nfs/lab/projects/pbmc_snATAC/analysis_v2/rasqual/'
sampdir  = '/nfs/lab/projects/pbmc_snATAC/analysis_v2/cell_type_bam/'
samples  = paste0("PBMC", c(1:6, 8:10,12:15))

### FIlter original vcf files for just  snps in peaks, samples of interest, imputation quality, heterozygous genotype and removing indels

In [6]:
filter_vcf = function(cell) {
        
    for (n in 1:22) {

    vcf    = paste0(vcf_dir, "/chr",n ,'.PBMC.topmed.hg19.final.vcf.gz')
    outdir = paste(homedir, reso, cell, "vcfs_peaksonly", sep="/")    
    system(paste("mkdir -p ",outdir  ) )   
    outvcf = paste0(outdir, "/chr",n ,'.filt.vcf')
    bed    = paste0(homedir, reso, "/",cell, "/" , cell, '.filtered.bed')
        
    system(paste('bcftools view', vcf ,
    '-R' , bed, 
    "-i \'INFO/R2[0]>0.7\' | bcftools view -g het --types snps -o" , outvcf))
}
    }

In [7]:
reso = "broad"

In [8]:
ids = c(1:6, 8:10,12:15)

In [11]:
celltypes =list.files(paste(homedir,reso, sep="/"))

In [13]:
celltypes = celltypes[!grepl(".txt", celltypes)]

In [9]:
celltypes = "t"

In [10]:
mclapply(celltypes, function(x) filter_vcf(x) , mc.cores = 32)

[[1]]
NULL

In [11]:
for (cell in celltypes) {
setwd(paste(homedir, reso, cell, "vcfs_peaksonly", sep="/"))
system("bcftools concat chr1.filt.vcf chr2.filt.vcf chr3.filt.vcf chr4.filt.vcf chr5.filt.vcf chr6.filt.vcf chr7.filt.vcf chr8.filt.vcf chr9.filt.vcf chr10.filt.vcf chr11.filt.vcf chr12.filt.vcf chr13.filt.vcf chr14.filt.vcf chr15.filt.vcf chr16.filt.vcf chr17.filt.vcf chr18.filt.vcf chr19.filt.vcf chr20.filt.vcf chr21.filt.vcf chr22.filt.vcf -Oz -o full.chr.vcf.gz")
#system("awk \'{if($0 !~ /^#/) print \"chr\"$0; else print $0}\' full.vcf > full.chr.vcf")
#system('bgzip full.chr.vcf -f')
system('tabix full.chr.vcf.gz -f')
    }

In [12]:
for (cell in celltypes){
new_vcf_dir = paste0(homedir, reso,"/" ,cell, "/vcfs_peaksonly" )    
bam = paste0(sampdir, "PBMC", ids,"/", reso, '/pbmc', ids, "_", cell, ".bam")
writeLines(bam, paste(new_vcf_dir, 'bam.list.txt', sep="/"))
    }

In [13]:
rasqual_ase = function (cell, reso){
setwd(paste0(homedir, reso,"/" ,cell, "/vcfs_peaksonly" )  )
system("rm full.ase*")   ## remove older files 
system('sh $RASQUALDIR/src/ASVCF/createASVCF.sh paired_end bam.list.txt full.chr.vcf.gz full.ase.vcf.gz atac')
system('/home/pbenaglio/bin/bcftools view full.ase.vcf.gz -i \'COUNT(GT=\"het\")>1\' -Oz -o full.ase.filtered.vcf.gz')
system('tabix full.ase.filtered.vcf.gz -f')
    }

In [14]:
paste0(homedir, reso)

[1] "/nfs/lab/projects/pbmc_snATAC/analysis_v2/rasqual/broad"

In [15]:
mclapply(celltypes, function (x) rasqual_ase(cell=x, reso=reso),mc.cores = 40)

[[1]]
[1] 0

In [16]:
homedir

[1] "/nfs/lab/projects/pbmc_snATAC/analysis_v2/rasqual/"

In [17]:
for (cell in celltypes){
setwd(paste0(homedir,reso ,"/", cell, "/vcfs_peaksonly"))
system( "zgrep \'#\' full.ase.filtered.vcf.gz > header") 
test = read.table('full.ase.filtered.vcf.gz')
for(n in 1:2){
for( i in 10:22){
tabs  = str_split_fixed(test[,i], "\\:", 5)
ac    = tabs[tabs[,1]=="0|1" | tabs[,1]=="1|0", 5]
acs   = str_split(ac,",")
acsp  = t(sapply(acs, sample))
acsp  = apply(acsp, 1, str_flatten, collapse=",")
tabs2 = tabs
tabs2[tabs2[,1]=="0|1" | tabs2[,1]=="1|0", 5] <- acsp

tt    =  apply(tabs2, 1, str_flatten, collapse=":")
   
    if (i ==10){
        TT = tt  }else{
        TT = cbind(TT,tt)}
    }

europ = c(2,3,4,5,7,8,9,10,11,12)
shuff = sample(europ)

if(n==1) {
    perm =  c(6, shuff[1:4], 13, shuff[5:10],1)
} else if(n==2) {
    perm =  c(13, shuff[1:4], 1, shuff[5:10],6)}
    
TT = TT[,perm]
testp = cbind(test[,1:9], TT)

outfile = paste0("full.ase.filtered.allelic.perm",n)
setwd(paste0(homedir,reso ,"/", cell, "/vcfs_peaksonly"))
write.table(testp, outfile, col.names=F, row.names=F, quote=F, sep="\t" )
system(paste0('cat header ', outfile, ' > ', outfile, '.vcf'))
system(paste0('bgzip ', outfile, '.vcf -f'))
system(paste0('tabix ', outfile, '.vcf.gz -f'))
system(paste("rm", outfile))    
    }
    }